In [ ]:
import os
import torch
import pandas as pd
from skimage import io, transform
import numpy as np
import matplotlib.pyplot as plt
from torch.utils.data import Dataset, DataLoader
#from torchvision import transforms, utils
import glob
import torch
from PIL import Image
import re
import matplotlib.pyplot as plt
import torchvision

In [ ]:
import sys
sys.path.append('../../torch_videovision/')

In [ ]:
from torchvideotransforms.video_transforms import Compose as vidCompose
from torchvideotransforms.video_transforms import Normalize as vidNormalize
from torchvideotransforms.volume_transforms import ClipToTensor

In [ ]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline

In [ ]:
plt.rcParams['figure.figsize'] = (20,10)
font = {'family' : 'DejaVu Sans',  'weight' : 'normal',  'size'  : 20}
plt.rc('font', **font)

In [ ]:
#from spatial_transforms import *
#from temporal_transforms import *

In [ ]:
import vidaug.augmentors as va

In [ ]:
# transform = transforms.Compose([
#     transforms.Resize((576, 704)),
#     transforms.RandomAffine(
#             degrees=15,
#             translate=(0.05, 0.05),
#             scale=(0.95, 1.05),
#             resample=2,
#             fillcolor=0,
#         ),
#         transforms.ColorJitter(
#             brightness=0.3,
#             contrast=0.2,
#             saturation=0.4,
#             hue=0.1
#         ),
#         transforms.RandomHorizontalFlip(),
#     transforms.ToTensor(),
#     transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
# ])

In [ ]:
video_transform_list = [
#     RandomRotation(30),
#     Resize((576, 704)),
#     ColorJitter(0.2,0.2,0.2,0.2),
    ClipToTensor(channel_nb=3),
    vidNormalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
]
tensor_transform = vidCompose(video_transform_list)

In [ ]:
transform = va.SomeOf([
    va.RandomRotate(degrees=20), #andomly rotates the video with a degree randomly choosen from [-10, 10]  
    va.HorizontalFlip(),# horizontally flip the video with 100% probability
    va.ElasticTransformation(0.1,0.1),
    va.GaussianBlur(sigma=0.1),
    va.InvertColor(),
    va.Superpixel(0.2,2),
    va.Multiply(2.0),
    va.Add(10),
    va.Pepper(),
    va.PiecewiseAffineTransform(0.3,0.3,0.3),
    va.Salt(),
    va.TemporalRandomCrop(size=16),
    #va.TemporalElasticTransformation(),
    #va.InverseOrder(),
], N=3)

In [ ]:
def parse_img_num(img):
    m = re.search(r"frame(\d+).png", img)
    if m:
        return f"{int(m.groups()[0]):06d}"
    print(img)
    raise Exception("Couldn't parse number")

In [ ]:
class MySampler(torch.utils.data.Sampler):
    def __init__(self, end_idx, seq_length):
        indices = []
        for i in range(len(end_idx) - 1):
            start = end_idx[i]
            end = end_idx[i + 1] - seq_length
            if start > end:
                pass
            else:
                indices.append(torch.arange(start, end))
        indices = torch.cat(indices)
        self.indices = indices
        
    def __iter__(self):
        indices = self.indices[torch.randperm(len(self.indices))]
        return iter(indices.tolist())
    
    def __len__(self):
        return len(self.indices)

In [ ]:
class MyDataset(Dataset):
    def __init__(self, image_paths, seq_length, transform, tensor_transform, length): #csv_file, 
        self.image_paths = image_paths
        self.seq_length = seq_length
        self.transform = transform
        self.tensor_transform = tensor_transform
        self.length = length
        
    def __getitem__(self, index):
        start = index
        end = index + self.seq_length
        print('Getting images from {} to {}'.format(start, end))
        indices = list(range(start, end))
        images = []
        tr = self.transform
        for i in indices:
            image_path = self.image_paths[i][0]
            image = Image.open(image_path)
#             if self.transform:
#                 image = tr(image)
            images.append(image)
        x = self.transform(images)
        x = self.tensor_transform(x)
#        x = torch.stack(images)
        y = torch.tensor([self.image_paths[start][1]], dtype=torch.long)
        x = x.permute(1,0,2,3)
        return x, y
    
    def __len__(self):
        return self.length

In [ ]:
# class MyDataset(Dataset):
#     def __init__(self, image_paths, seq_length, transform, length): #csv_file, 
#         self.image_paths = image_paths
#         self.seq_length = seq_length
#         self.transform = transform
#         self.length = length
        
#     def __getitem__(self, index):
#         start = index
#         end = index + self.seq_length
#         print('Getting images from {} to {}'.format(start, end))
#         indices = list(range(start, end))
#         images = []
#         tr = self.transform
#         for i in indices:
#             image_path = self.image_paths[i][0]
#             image = Image.open(image_path)
#             if self.transform:
#                 image = tr(image)
#             images.append(image)
# #        x = video_transform(images)
#         x = torch.stack(images)
#         y = torch.tensor([self.image_paths[start][1]], dtype=torch.long)
#         return x, y
    
#     def __len__(self):
#         return self.length

In [ ]:
#root_dir = '/media/raid/astamoulakatos/nsea_frame_sequences/centre_Ch2/'
root_dir = '/media/scratch/astamoulakatos/nsea_video_jpegs/'
class_paths = [d.path for d in os.scandir(root_dir) if d.is_dir]

In [ ]:
class_paths

In [ ]:
df = pd.read_csv('../small_dataset_csvs/events_with_number_of_frames_stratified.csv')
df.head()

In [ ]:
len(df)

In [ ]:
len(df[df.number_of_frames>=48])

In [ ]:
len(df[df.fold!=0])

In [ ]:
df_new = df[df.number_of_frames>=48]; df_new

In [ ]:
class_names = ['exp_fs','bur','exp','exp_and','exp_fj']
one_hot_classes = [[1,0,0,0,1],[0,1,0,0,0],[1,0,0,0,0],[1,0,0,1,0],[1,0,0,1,0]]
class_image_paths = []
end_idx = []
for c, class_path in enumerate(class_paths):
     for d in os.scandir(class_path):
        if d.is_dir:
            if d.path in df[df.fold==0].event_path.values:
                paths = sorted(glob.glob(os.path.join(d.path, '*.png')))
                # Add class idx to paths
                paths = [(p, one_hot_classes[c]) for p in paths]
                class_image_paths.extend(paths)
                end_idx.extend([len(paths)])

In [ ]:
'exp_fj' in class_path

In [ ]:
df[df.fold!=0].event_path.count()

In [ ]:
df.event_path.count()

In [ ]:
df[df.fold==0].event_path.count()

In [ ]:
end_idx = [0, *end_idx]
end_idx = torch.cumsum(torch.tensor(end_idx), 0)

In [ ]:
end_idx

In [ ]:
seq_length = 16

In [ ]:
sampler = MySampler(end_idx, seq_length)

In [ ]:
dataset = MyDataset(
    image_paths=class_image_paths,
    seq_length=seq_length,
    transform=transform,
    tensor_transform=tensor_transform,
    length=len(sampler))

In [ ]:
loader = DataLoader(
    dataset,
    batch_size=2,
    sampler=sampler,
    drop_last=True,
    num_workers=0
)

In [ ]:
def imshow(inp, title=None):
    inp = inp.numpy().transpose((1, 2, 0))
    mean = np.array([0.485, 0.456, 0.406])
    std = np.array([0.229, 0.224, 0.225])
    inp = std * inp + mean
    inp = np.clip(inp, 0, 1)
    plt.figure(figsize=(30,30))
    plt.imshow(inp)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)  # pause a bit so that plots are updated

In [ ]:
# Get a batch of training data
inputs, classes = next(iter(loader))
inputs = inputs.squeeze(dim = 0)

# Make a grid from batch
out = torchvision.utils.make_grid(inputs)


for i, f in enumerate(one_hot_classes):
    if np.array_equal(classes[0][0].numpy(), np.asarray(f)):
        title = class_names[i]
        
        
imshow(out, title=title)

In [ ]:
inputs, classes = next(iter(loader))
inputs = inputs.permute(0,2,1,3,4)
inputs = inputs.squeeze(dim = 0)

# Make a grid from batch
class_names = ['exp_fs','bur','exp','exp_and','exp_fj']
one_hot_classes = [[1,0,0,0,1],[0,1,0,0,0],[1,0,0,0,0],[1,0,0,1,0],[1,0,0,1,0]]
out = torchvision.utils.make_grid(inputs)
for i, f in enumerate(one_hot_classes):
    if np.array_equal(classes[0].numpy(), np.asarray(f)):
        title = class_names[i]
imshow(out, title=title)

In [ ]:
out.shape

In [ ]:
inputs.shape

In [ ]:
inputs = inputs.squeeze(dim = 0)

In [ ]:
inputs = inputs.permute(0,2,1,3,4)

In [ ]:
classes